In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [19]:

main_webUrl = "https://www.rumah123.com/jual/jawa-tengah/rumah/?page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [20]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [21]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [22]:
dataframe = {}
dataframe["Nama"] = []
dataframe["Harga"] = []

In [23]:
for i in range(1, 200):
    webUrl = f"https://www.rumah123.com/jual/jawa-tengah/rumah/?page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe.keys():
                dataframe[prop_key] = []
            
            if prop_key in dataframe.keys():
                dataframe[prop_key].append(prop_value)
            
        print("")


/properti/semarang/hos18666001/

/properti/semarang/hos18244386/

/properti/semarang/hos17415303/

/properti/semarang/hos19287900/

/properti/semarang/hos9707351/

/properti/semarang/hos19207289/

/properti/semarang/hos19203886/

/properti/salatiga/hos19169434/

/properti/semarang/hos18615076/

/properti/banyumas/hos19275532/

/properti/sukoharjo/hos19271766/

/properti/semarang/hos19173613/

/properti/semarang/hos17320563/

/properti/semarang/hos18268184/

/properti/semarang/hos17177773/

/properti/semarang/hos19390600/

/properti/tegal/hos16627518/

/properti/tegal/hos17806349/

/properti/semarang/hos19392902/

/properti/semarang/hos19392170/

/properti/semarang/hos19340202/

/properti/purwokerto/hos18339402/

/properti/semarang/hos18395983/

/properti/solo/hos11728810/

/properti/solo/hos6384106/

/properti/solo/hos17709952/

/properti/solo/hos14270205/

/properti/solo/hos14270205/

/properti/semarang/hos17827811/

/properti/semarang/hos19238690/

/properti/semarang/hos14206068/

/p

In [24]:
rumah123_df = pd.DataFrame.from_dict(dataframe, orient='index').T
rumah123_df = rumah123_df.fillna(0)
rumah123_df.to_json("Provinsi_JawaTengah.json", orient="records")

In [25]:
rumah123_df

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Jumlah Lantai,Kondisi Properti
0,Dijual Rumah 2 Lantai Di Bromo Gajahmungkur Se...,Rp 30 Miliar,6,4,1353 m²,800 m²,2,SHM,4400 Watt,1,2,1,2,Baru
1,Rumah Hitung Tanah Lokasi Strategis di Candi B...,Rp 20 Miliar,7,3,1250 m²,500 m²,1,SHM,6600 Watt,2,1,3,1,Bagus
2,"Jual Cepat Rumah 2 Lantai Besar Murah, SHM, Te...","Rp 2,5 Miliar",5,6,244 m²,259 m²,3,SHM,3300 Watt,1,1,3,2,Sudah Renovasi
3,"Rumah Super Mewah di Puri Arga Golf, BSB City ...",Rp 20 Miliar,7,7,800 m²,900 m²,2,SHM,Lainnya,1,1,1,2,Bagus
4,Rumah Tengah Kota Pinggir Jalan Dijual Cepat,"Rp 6,5 Miliar",5,1,363 m²,399 m²,2,SHM,2200 Watt,1,1,1,2,Bagus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,Rumah Murah Siap Huni di Sendangmulyo Tembalang,Rp 750 Juta,0,0,160 m²,80 m²,0,SHM,Lainnya,0,0,0,0,0
1895,Rumah Kawasan Elit Pinggir Jalan Raya Semeru K...,"Rp 8,5 Miliar",0,0,850 m²,325 m²,0,SHM,1300 Watt,0,0,0,0,0
1896,Rumah 2 Lantai di Indrapasta\nSemarang tengah ...,"Rp 7,25 Miliar",0,0,334 m²,320 m²,0,SHM,0,0,0,0,0,0
1897,Rumah di di Puri Anjasmoro Semarang Barat dek...,Rp 12 Miliar,0,0,624 m²,800 m²,0,SHM,0,0,0,0,0,0
